In [81]:
# Dependencies

from bs4 import BeautifulSoup as bs
import pandas as pd
import pymongo
import requests
!which chromedriver
import time

/usr/local/bin/chromedriver


In [82]:
# Import Splinter and set the chromedriver path
from splinter import Browser
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

# NASA Mars News


In [83]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

In [84]:
# Retrieve page with the requests module
response = requests.get(url)

In [85]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

In [86]:
# Extract title text
news_title = soup.title.text
print(news_title)

News  – NASA’s Mars Exploration Program 


In [87]:
# Print all paragraph texts
paragraphs = soup.find_all('p')
for paragraph in paragraphs:
    print(paragraph.text)

Managed by the Mars Exploration Program and the Jet Propulsion Laboratory for NASA’s Science Mission Directorate


# JPL Mars Space Images - Featured Images

In [92]:
# Visit the url for JPL Featured Space Image
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)
browser.find_by_id('full_image').click()

In [93]:
featured_image_url = browser.find_by_css('.fancybox-image').first['src']
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18791_ip.jpg'

# Mars Weather

In [ ]:
# URL of page to be scraped
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

In [ ]:
html = browser.html
soup = bs(html, 'html.parser')

In [ ]:
# loop through every tweet you grab (findAll)
tweets = soup.body.find_all('p', 'tweet-text')

for tweet in tweets:
# if the tweet contains "Sol" we know it is a tweet about weather
   if 'Sol' in tweet.text:
       mars_weather = tweet.text
       break

In [94]:
mars_weather

'Sol 2171 (2018-09-14), high -12C/10F, low -65C/-84F, pressure at 8.79 hPa, daylight 05:43-17:59'

# Mars Facts

In [ ]:
# URL of page to be scraped
url = 'https://space-facts.com/mars/'

In [ ]:
tables = pd.read_html(url)
tables

In [ ]:
# Now let's slice off the dataframe using normal indexing.
mars_df = tables[0]
mars_df.columns = ['description', 'value']
mars_df.set_index('description', inplace=True)
mars_df

In [ ]:
#Converting the data to a HTML table string and stripping unwanted newlines to clean up the table

mars_html = mars_df.to_html()
mars_html.replace("\n", " ")

# Mars Hemispere

In [ ]:
# Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
base_url = 'https://astrogeology.usgs.gov'
browser.visit(usgs_url)

In [ ]:
#Saving browser in html
mars_hem_html = browser.html

In [ ]:
#Scraping browser into soup and using soup to find links to full resolution images of mars hemispheres
mars_hem_soup = bs(mars_hem_html, 'lxml')
images_links = mars_hem_soup.find_all('div', class_='item')

In [ ]:
#Creating a list to store dictionaries of data for each hemisphere
hemisphere_image_urls = []
images_url = []

In [ ]:
#Loop through images_links to find href and concatinate them with the base_url to get full link
for image in images_links:
    hemisphere_dict = {} 
    image_href = image.find('a', class_='itemLink product-item')
    h_link = base_url + image_href['href']
    browser.visit(h_link)
    time.sleep(1)
    hemisphere_html = browser.html
    hemisphere_soup = bs(hemisphere_html, 'lxml')
    
    img_title = hemisphere_soup.find('div', class_='content').find('h2', class_='title').text
    hemisphere_dict['title'] = img_title
    
    img_url = hemisphere_soup.find('div', class_='downloads').find('a')['href']
    hemisphere_dict['img_url'] = img_url
    images_url.append(img_url)
    
    # Append dictionary to list
    hemisphere_image_urls.append(hemisphere_dict)
      
hemisphere_image_urls

In [ ]:
import shutil
import requests
for img in images_url:
    response = requests.get(img, stream=True) 
    with open('img.jpg', 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)

In [95]:
# Display the image with IPython.display
from IPython.display import Image

Image(url='img.jpg')